In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4

cpu


In [2]:
text_file = "fcc-gpt-course/wizard_of_oz.txt"

with open(text_file, "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(set(text))
print(chars)

vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
# tokenizers

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }

# encoder
encode = lambda s: [string_to_int[c] for c in s]
# decoder
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [4]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    print(ix)

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    x, y = x.to(device), y.to(device)

    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

tensor([ 39791, 169216, 177193, 134543])
inputs:
tensor([[62, 54, 65, 65, 78,  1, 69, 65],
        [76, 58, 65, 65, 58, 57,  1, 57],
        [58, 75, 58, 71,  1, 57, 54, 71],
        [57,  1, 61, 54, 75, 58,  1, 55]])
targets:
tensor([[54, 65, 65, 78,  1, 69, 65, 58],
        [58, 65, 65, 58, 57,  1, 57, 71],
        [75, 58, 71,  1, 57, 54, 71, 58],
        [ 1, 61, 54, 75, 58,  1, 55, 58]])


In [5]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [6]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index) 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)

            # apply softmax to get the probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

print(generated_chars)


c-kV GJ40﻿Zin(k.3[fQe3Xm_Wo﻿.c6Fp8ZMYn!_m﻿O
hE8q1T(cGsRIbhF(z(YXGaaVufbs8nNkDR2s﻿ouGY*5u-?veP)P&]l&2hz0(TW﻿6
P&:Pnn
rYMS?.MC
rDF((
Xi﻿N93QsIur-U_(WG1qI;XRNU8Sm1S?"-?SHFu[OLDY;!3zLMB4i:
2*3LlE0[uAH6MS?.W;QGItv5][0wfq5.u1qjinM!Eq?2.Gbx-K?5IMC]d5VXE,*;n]lwPP'[JN Q"-sLk8YUqe-1p:WVdMnW*R([i9x4_6D]Wv5zzfh0!*T(ojPVtvC42KaaYn57aG3*sf&Z;,aIYNrvv"qAO"Vn&wk(nw76p?UoEJfIMSE,H6GyGnJSmj.284iZ-kH(mXm)wmA;*"
cnN9:Dg76
dJ!m
mvBzCk276 -DY?mgQSo]I;v3;ddaPp:B*5mDI*7qXVQ[1ouK6jN*IV VS:)Jr&w8Z,4J41KUfU!&_0,HtIMceZhCT
